In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from pinecone import Pinecone, ServerlessSpec

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)

INDEX_NAME="demo-index"
if INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(name=INDEX_NAME, dimension=768, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"))

In [ ]:
from langchain_ollama import OllamaEmbeddings 
OLLAMA_HOST = os.getenv("OLLAMA_HOST")

embeddings = OllamaEmbeddings(model="nomic-embed-text", base_url=OLLAMA_HOST)

http://ollama:11434


In [15]:
from langchain_pinecone import PineconeVectorStore

index = pc.Index(INDEX_NAME)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

Index host ignored when initializing with index object.


In [16]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("technical.pdf")
documents = loader.load()

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators = ["\n\n","\n"," "])
splitted_documents = text_splitter.split_documents(documents)

In [18]:
# vector_store.add_documents(splitted_documents)

In [19]:
from langchain_core.prompts import PromptTemplate

template = """
    You are an AI assitant.
    If the answer isn't in the context, say you don't know
    Context: {context}
    Question: {question}
    Answer:
"""

prompt = PromptTemplate(template = template, input_variables = ["context", "question"])

In [20]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model = "llama3.2:1b", temperature = 0.5, base_url = OLLAMA_HOST)

In [21]:
from langchain_classic.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    chain_type_kwargs = {
        "prompt": prompt
    },
    retriever = vector_store.as_retriever(search_kwargs = { "k": 2 })
)

In [ ]:
query = "What's the plan?"

result = qa_chain(query)
# result = qa_chain.invoke({ "query": query })
print(result["result"])